# **0. 필요한 라이브러리 import**

hugging face 의 모델들을 사용하기 위해서 아래와 같은 모듈들을 import 해준다.

**transformers** : Hugging Face에서 제공하는 라이브러리로, 다양한 사전 학습된 트랜스포머 모델(GPT, BERT, T5 등)을 쉽게 불러오고 파인튜닝할 수 있도록 지원한다.
주요 기능: 모델 로드, 토크나이저, 학습, 추론

**datasets** : Hugging Face에서 제공하는 대규모 데이터셋 라이브러리로.
datasets.load_dataset()을 이용해서 다양한 공개 데이터셋(KorQuAD, SQuAD, IMDB 등)을 쉽게 불러올 수 있으며.
데이터 로딩, 전처리, 변환, 배치 단위 처리 등을 지원한다.

**wandb** : 머신러닝 실험을 추적, 로깅하고 시각화할 수 있는 도구로
학습 과정에서 손실(loss), 정확도(accuracy) 등을 저장하고 대시보드에서 확인할 수 있다.
wandb.init()을 사용해서 프로젝트를 생성하고 학습 로그를 관리할 수 있다.

In [ ]:
!pip install transformers datasets wandb

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: http://repo.ai.gato/registry/repository/pypi-proxy/simple
  Using cached http://repo.ai.gato/registry/repository/pypi-proxy/packages/fsspec/2024.12.0/fsspec-2024.12.0-py3-none-any.whl (183 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.12.0 requires numba-cuda<0.0.18,>=0.0.13, which is not installed.
s3fs 2025.2.0 requires fsspec==2025.2.0.*, but you have fsspec 2024.12.0 which is incompatible.
cudf-cu12 24.12.0 requires pyarrow<19.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 19.0.1 which is incompatible.


# **1. 데이터 전처리**

KorQuAD 데이터셋을 로드하고 KLUE-BERT를 사용하여 토큰화한 후, 훈련을 위한 전처리를 수행하는 과정이다.  

### **전처리 과정**
1. **context + question을 BERT 토크나이저로 토큰화** (**512 토큰 제한**)  
2. **정답(Answer)의 문자 위치 → 토큰 위치로 변환**  
3. **start_positions, end_positions를 추가하여 모델이 훈련할 수 있도록 변환**  

> 나는 **Encoder 기반의 BERT 모델**을 사용하기 때문에, `answer`를 생성하는 **생성형(Generative) 모델**들과는 달리, 문장의 **시작, 끝 위치를 예측하는 방식**으로 모델을 구성했다.


In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer

# 데이터셋 로드
dataset = load_dataset("KorQuAD/squad_kor_v1")

# 사용할 사전 학습 모델 선택 (KLUE BERT 사용)
model_name = "klue/bert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 전처리 함수 정의
def preprocess_function(examples):
    inputs = tokenizer(
        examples["context"],
        examples["question"],
        truncation=True,
        padding="max_length",
        max_length=512,
    )

    # 정답(Answer)의 시작 위치 찾기
    start_positions = []
    end_positions = []

    for i, answer in enumerate(examples["answers"]):
        if len(answer["text"]) == 0:  # 정답이 없는 경우
            start_positions.append(0)
            end_positions.append(0)
        else:
            start_char = answer["answer_start"][0]  # 정답의 시작 위치 (문자 기준)
            end_char = start_char + len(answer["text"][0])  # 정답의 끝 위치

            # 정답이 `input_ids` 내 어느 토큰에 해당하는지 찾기
            token_start_index = inputs.char_to_token(i, start_char)
            token_end_index = inputs.char_to_token(i, end_char - 1)

            if token_start_index is None or token_end_index is None:
                start_positions.append(0)
                end_positions.append(0)
            else:
                start_positions.append(token_start_index)
                end_positions.append(token_end_index)

    # 최종 데이터셋 반환
    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs


# 데이터셋 전처리 적용
tokenized_datasets = dataset.map(preprocess_function, batched=True)
# 데이터 형식 변환 (Trainer가 사용할 수 있도록 설정)
tokenized_datasets.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "start_positions", "end_positions"]
)

# **2. 훈련 및 평가**

이 코드는 Trainer를 사용하여 KLUE-BERT를 KorQuAD 데이터셋에 파인튜닝하는 과정으로 구성을 보면 크게 3단계로 나뉜다.
1. 질문-답변(Task)에 특화된 BERT 모델을 자동으로 로드한다.
2. trainingArguments를 사용해 훈련 설정 (학습률, 배치 크기, 로깅 등)
3. Trainer를 사용해 모델, 데이터셋, 토크나이저 설정 후 학습 준비 완료

**HuggingFace** 가 제공하는 아주 간편한 메서드를 통해 모델을 쉽게 불러올 수 있었는데, AutoModelForQuestionAnswering 을 통해 모델을 불러오게 되면, 모델 구조가 자동으로 QA task를 위해 맞춰지며, 기존 BERT 모델에 HEAD를 붙히는 식으로 구현된다. 즉 BACKBORN 을 유지하면서, 출력을 낼 수 있는 HEAD를 자동으로 붙히는

In [ ]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

# 모델 불러오기
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

# 훈련 설정
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=500,
    save_total_limit=2,  # 최근 2개 모델만 저장
    load_best_model_at_end=True,  # 가장 좋은 모델 저장
    report_to="wandb"  # W&B 로깅
)

# Trainer 정의
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer
)


Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/gatoai/python/venv/3.10/lib/python3.10/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_7203/608393248.py:24: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Trainer 클래스는 Hugging Face에서 제공하는 자동 학습 관리 도구로, 앞서 trainer를 만들 때 설정했던 설정 값들을 토대로 trainer 클래스가 학습하게 된다.

In [ ]:
# 모델 학습 시작
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: kkk121026 (kkk121026-kookmin-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,0.487700,0.512494
2,0.317100,0.554175
3,0.207800,0.614121
4,0.139400,0.799269
5,0.077400,0.965911


TrainOutput(global_step=37755, training_loss=0.2602638837723144, metrics={'train_runtime': 8434.4087, 'train_samples_per_second': 35.81, 'train_steps_per_second': 4.476, 'total_flos': 7.892076592075776e+16, 'train_loss': 0.2602638837723144, 'epoch': 5.0})

# **3. 학습 종료 및 평가**

학습 이후 `trainer.evaluate()`를 통해 **validation set**에 대한 결과를 볼 수 있으며,  
학습된 모델의 **가중치**와 **토크나이저**도 함께 저장하게 된다.  

단순히 **loss 값이 낮다**고 해서 모델이 잘 작동한다고 단정할 수 없으므로,  
실제로 모델이 **예측을 얼마나 잘하는지** 확인하기 위해 **임의의 데이터를 생성하여 추론을 수행**했다.  

실험 결과, 아래 예시를 포함하여 대부분의 질문에 대해 **모델이 올바르게 추론하는 것을 확인할 수 있었다.** 🚀  

---

## **실험 결과 예시**

### 🔹 **예제 1**
- **Context:**  
  > *허깅페이스는 자연어 처리(NLP) 및 머신러닝을 위한 플랫폼이다. 주요 제품으로는 트랜스포머 라이브러리가 있다.*
- **Question:**  
  > *허깅페이스의 주요 제품은 무엇인가?*
- **Answer (Ground Truth):**  
  > *트랜스포머 라이브러리*
- **Predict (모델 예측):**  
  > *트랜스포머 라이브러리*

---

### 🔹 **예제 2**
- **Context:**  
  > *아이작 뉴턴(Isaac Newton)은 17세기 영국의 수학자이자 물리학자로, 미적분학과 고전역학의 기초를 다졌다. 그의 저서 《프린키피아(Philosophiæ Naturalis Principia Mathematica)》는 물리학 역사에서 가장 영향력 있는 책 중 하나로 평가받는다*
- **Question:**  
  > *아이작 뉴턴은 어떤 학문에서 중요한 기여를 했는가?*
- **Answer (Ground Truth):**  
  > *미적분학과 고전역학*
- **Predict (모델 예측):**  
  > *미적분학과 고전역학*

---

### 🔹 **예제 2**
- **Context:**  
  > *대한민국의 수도는 서울이며, 1천만 명이 넘는 인구가 거주하는 대도시이다. 서울은 정치, 경제, 문화의 중심지로서 다양한 역사적 유산과 현대적인 건축물이 공존하는 곳이다*
- **Question:**  
  > *대한민국의 수도는 어디인가?*
- **Answer (Ground Truth):**  
  > *서울*
- **Predict (모델 예측):**  
  > *서울*

---


✔ **추가 실험에서도 대부분 정확한 답변을 도출할 수 있었음!**  
📌 **하지만 직관적으로 답이 주어지지 않는 일부 어려운 질문에서는 부정확한 답변을 생성하는 경우도 있었다.**


In [ ]:
eval_results = trainer.evaluate()
print(f"Evaluation Results: {eval_results}")

Evaluation Results: {'eval_loss': 0.5124937295913696, 'eval_runtime': 40.4461, 'eval_samples_per_second': 142.758, 'eval_steps_per_second': 17.851, 'epoch': 5.0}


In [ ]:
save_path = "./saved_model"
trainer.save_model(save_path)  # 모델 가중치 저장
tokenizer.save_pretrained(save_path)  # 토크나이저도 저장

print(f"Model saved at {save_path}")

Model saved at ./saved_model


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

model_name = "klue/bert-base"

context = "기침소리를 낸 것은 영의정이다."
question = "누가 기침소리를 내었는가?"
expected_answer = "영의정"  # 실제 정답


device = "cuda" if torch.cuda.is_available() else "cpu"
inputs = tokenizer(context, question, return_tensors="pt").to(device)

with torch.no_grad():
    outputs = model(**inputs)
    start_logits = outputs.start_logits
    end_logits = outputs.end_logits

# 가장 높은 확률을 가진 start, end 인덱스 찾기
start_idx = torch.argmax(start_logits)
end_idx = torch.argmax(end_logits)

predicted_answer = tokenizer.convert_tokens_to_string(
    tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][start_idx:end_idx+1])
)

# 결과 출력
print(f"Question: {question}")
print(f"Expected Answer: {expected_answer}")
print(f"Predicted Answer: {predicted_answer}")


Question: 누가 기침소리를 내었는가?
Expected Answer: 영의정
Predicted Answer: 영의정


## KorQuAD (Korean Question Answering Dataset) 소개

KorQuAD(Korean Question Answering Dataset)는 한국어 기계 독해(Machine Reading Comprehension, MRC) 태스크를 위한 데이터셋으로, Hugging Face에서는 `"KorQuAD/squad_kor_v1"`라는 이름으로 제공되고 있다.

---

### 🔹 1. KorQuAD란?
KorQuAD는 한국어 질문-답변 태스크(QA, Question Answering)를 학습하고 평가하기 위해 구축된 데이터셋이다. 이 데이터셋은 영어의 대표적인 QA 데이터셋인 **SQuAD**(Stanford Question Answering Dataset)를 참고하여 만들어졌다.

- **KorQuAD 1.0**: 한국어 Wikipedia 문서에서 추출된 문단을 기반으로 구성됨.
- **KorQuAD 2.0**: 추가적인 노이즈 데이터를 포함하여 질문이 더 어려워진 버전이지만, Hugging Face에서는 **1.0 버전만 제공**된다.

---

### 🔹 2. "KorQuAD/squad_kor_v1" 데이터셋 구조
Hugging Face의 `"KorQuAD/squad_kor_v1"`은 **SQuAD 형식을 따른다**.
데이터는 크게 `train`과 `validation`으로 구성되어 있다.

#### ✔ 데이터셋 크기

| 데이터셋 | 샘플 개수 |
|----------|-----------|
| Train (훈련 데이터) | 약 60,407개 |
| Validation (검증 데이터) | 약 5,774개 |

---

### 🔹 3. 데이터셋 상세 구조
각 데이터 샘플은 다음과 같은 필드를 포함한다.

#### ✔ 데이터 예시
```
{
    'id': '654321',  
    'title': '세종대왕',
    'context': '세종대왕은 조선의 네 번째 왕으로, 한글을 창제한 것으로 유명하다. 그는 1397년에 태어나 1418년에 즉위하였다...',
    'question': '세종대왕이 즉위한 연도는?',
    'answers': {'text': ['1418년'], 'answer_start': [50]}
}
```

#### ✔ 각 필드 설명

- **id**: 질문 샘플의 고유 ID
- **title**: 해당 문단의 제목
- **context**: 질문에 대한 정답이 포함된 문맥(Paragraph)
- **question**: 주어진 `context`를 바탕으로 한 질문
- **answers**: 정답 (정답 문자열과 해당 문자열의 시작 위치 포함)
  - `answers["text"]`: 실제 정답 텍스트
  - `answers["answer_start"]`: `context` 내에서 정답이 시작하는 문자 인덱스

---

### 🔹 4. KorQuAD를 사용하는 이유
**한국어 기반 QA 모델을 훈련할 수 있음**
**Hugging Face에서 쉽게 로드 가능**
**SQuAD 형식과 동일하여 기존 영어 기반 모델을 한국어로 전이 학습하기 좋음**

---


# **분석 및 고찰**

## **BERT 모델을 사용한 이유**

### **1. 사전 학습된 강력한 인코더 기반 모델**
BERT(Bidirectional Encoder Representations from Transformers)는 대규모 코퍼스를 활용한 사전 학습을 통해 강력한 문맥적 이해 능력을 갖추고 있다. 이를 통해 QA(Task)에서 높은 성능을 발휘할 수 있다.

### **2. 양방향 학습(Bidirectional Context Learning)**
BERT는 문장의 양방향 문맥을 동시에 고려하여 단어의 의미를 학습한다. 이는 질문과 문맥(Context) 간의 관계를 보다 정밀하게 이해하는 데 도움이 된다.

### **3. 토큰 수준의 정밀한 위치 예측**
KorQuAD와 같은 MRC(Task)에서는 정답의 시작과 끝 위치를 예측해야 한다. BERT는 각 토큰의 표현을 효과적으로 학습하며, 이를 통해 높은 정확도로 정답의 위치를 찾아낼 수 있다.

### **4. 사전 학습 + 파인튜닝 구조**
BERT는 이미 방대한 데이터로 사전 학습된 상태에서 제공되므로, 특정 도메인(예: 한국어 QA)에서 적은 양의 데이터로도 효과적인 파인튜닝이 가능하다.

### **5. 한국어 지원 모델(KLUE-BERT)의 존재**
KorQuAD 데이터셋을 활용하려면 한국어에 특화된 모델이 필요하다. KLUE-BERT는 한국어 자연어 처리(NLP)를 위해 최적화된 모델로, KorQuAD 태스크에서 강력한 성능을 낼 수 있다.

---

## **실험 결과 분석**

### **1. 학습 진행 과정**
- 학습 초기 부터, pre_trained 모델이 너무 잘 만들어 졌는지, 좋은 성능을 볼 수 있다.

### **2. 평가 결과 (Evaluation Results)**
- 모델이 단순한 질문뿐만 아니라, 복잡한 질문에서도 어느 정도 일반화 능력을 갖추었음을 확인하며, loss curve 또한 잘 떨어짐을 확인.

---

## **한계점 및 개선 방향**

### **1. Answer 생성 문제**
- 우리 모델은 기존의 정보를 기억하거나, 학습하려는 시도를 전혀 하지 않기 때문에
content 맥락의 여부에 크게 의존한다.

### **2. 추론 문제 **
- BERT 기반 모델은 전혀 생성하지 않고, 단순히 context 내의 어느 부분이 가장 중요한지 여부를 통해 시작과 끝점을 탐색하는 것이 전부이다. 즉 그렇기에 아예 context에서 추측해야 하는 어려운 문제의 경우는 적합한 답을 이끌어내지 못한다.
---

## **결론**
본 실험을 통해 KLUE-BERT 모델이 KorQuAD 데이터셋에서 효과적으로 학습될 수 있음을 확인하였다. 하지만, Encoder 기반의 모델이기 때문에 항상 Context 내에 정답이 있음을 보장해야 한다는 문제가 있다. 또한, 다양한 한국어 QA 데이터셋을 활용하여 보다 넓은 범위의 질의응답 태스크에 적용할 수 있을 것이다.


# **특이사항**  

내가 시도해본 접근 방식은 총 두 가지다.  

1. **BERT 기반 모델을 이용해 시작(Start), 끝(End) 토큰 위치 찾기**  
2. **Decoder 기반 모델을 이용하여 정답을 직접 생성하기**  

결과적으로 1번 방법을 선택하여 과제를 제출했는데, 두 방법의 장단점은 다음과 같다.  

---

## **1번 방법: BERT 기반 토큰 위치 예측**  

### **장점**  
- **학습이 비교적 빠름** → 사전 학습된 BERT 모델을 파인튜닝하기 때문에 학습 속도가 상대적으로 빠르다.  
- **간편한 구현** → Hugging Face에서 `AutoModelForQuestionAnswering`을 제공하므로 별도의 Head를 직접 구성할 필요 없이 불러와서 사용하면 된다.  
- **정확한 정답을 예측** → 정답이 `context` 내에 존재하기만 하면, 높은 정확도로 정답 위치를 찾아낼 수 있다.  

### **단점**  
- **Context 의존성이 높음** → 정답을 찾기 위해 반드시 `context`가 주어져야 하며, `context` 없이 단독 질문에 대해 답변할 수 없다.  
- **추론이 제한적** → 질문이 복잡하거나 `context` 내에서 명확한 정답을 찾기 어려운 경우 성능이 저하될 수 있다.  
- **정확한 위치 예측이 필요** → 정답의 시작과 끝 위치를 잘못 예측하면 답변이 부정확해질 수 있다.  

---

## **2번 방법: Decoder 기반 생성 모델**  

### **장점**  
- **Context 없이도 답변 가능** → 정답이 `context`에 포함되지 않더라도, 모델이 새로운 답변을 생성할 수 있다.  
- **보다 유연한 질의응답** → 단순한 `Span Extraction` 방식이 아니라, 열린 도메인의 질문에도 답할 수 있는 가능성이 있다.  

### **단점**  
- **학습이 오래 걸림** → 생성 모델(예: T5, GPT)은 일반적으로 학습 파라미터가 많아, 파인튜닝 시 시간이 오래 걸린다.  
- **정확성 문제** → `context`에 정답이 존재함에도 불구하고, 생성 과정에서 잘못된 답변을 만들어낼 가능성이 있다.  
- **추론 속도가 느림** → 답변을 생성하는 과정에서 디코딩이 필요하므로, `Span Extraction` 방식보다 추론 속도가 느리다.  

---

## **결론 및 선택 이유**  

BERT 기반의 **토큰 위치 예측 방식(1번)**을 선택한 이유는 다음과 같다.  

1. **효율적인 학습** → 제한된 시간 내에 최적의 성능을 내기 위해 학습 속도가 빠른 모델을 선택해야 했다.  
2. **Hugging Face 지원** → 이미 제공되는 QA 모델 구조를 활용하면, 모델 구조를 직접 수정하지 않고 빠르게 적용할 수 있었다.  
3. **높은 신뢰성** → 정답이 `context`에 포함되어 있기만 하면, 비교적 안정적인 성능을 보였다.  
4. **추론 속도** → 생성 모델보다 빠르게 예측할 수 있어 실제 응용에도 적합했다.

더불어 DLPC 환경에서의 Decoder 기반의 모델 같이 큰 모델을 학습하기에는 중간에 학습이 종료되어 처음부터 다시 돌려야 한다는 등 제약사항이 많아서 결국 BERT 기반의 모델로 제출하게 되었다.

하지만, 열린 도메인 질문(예: `"세계에서 가장 높은 산은?"`)에 대한 답변을 생성하는 것은 BERT 기반 모델로는 어려웠다. 따라서, 만약 특정 도메인에 한정되지 않은 QA 시스템을 구축해야 한다면, **Decoder 기반 모델**을 고려할 필요가 있다고 생각한다.

